In [1]:
# My Standard Spark Session!

# Python libraries:
import os
import sys
import re
from dateutil import parser
# import datetime
from datetime import datetime
from datetime import date
import builtins
import json
import functools
import operator
from itertools import product

# Numpy & Pandas!
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format
pd.options.display.max_columns = None

# Spark!
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
from pyspark.sql import SparkSession, Row


spark = SparkSession.builder.appName("myapp").getOrCreate()

#     spark = SparkSession.builder.master("yarn")\
#     .config("spark.executor.instances", "32")\
#     .config("spark.executor.cores", "4")\
#     .config("spark.executor.memory", "4G")\
#     .config("spark.driver.memory", "4G")\
#     .config("spark.executor.memoryOverhead","4G")\
#     .config("spark.yarn.queue","Medium")\
#     .appName("myapp")\
#     .getOrCreate()

sc = spark.sparkContext
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")
spark.conf.set("spark.debug.maxToStringFields","true")

In [2]:
%load_ext autoreload
%autoreload 2
# The autoreload extension is already loaded. To reload it, use:
#  %reload_ext autoreload


# mylib:
my_library = os.path.expanduser('~/.myconfigs')
my_spark = os.path.expanduser('~/spark2_dfanalysis')
sys.path.append(my_library)
sys.path.append(my_spark)


from shared.app_context import *
from builder.DataFrameBuild import *

ctx = ApplicationContext("Dev-Job")

DFB = DataFrameBuild(ctx.spark)

print("%16s  %s" % ("Python Version:",sys.version))
print("%16s  %s" % ("Python Path:",os.path.dirname(sys.executable)))
print("%16s  %s" % ("My Python Libs:",my_library))
print("%16s  %s" % ("My Spark Dir:",my_spark))
print("%16s  %s" % ("My Spark Ctx:",ctx.spark))
# print(ctx.spark)
# print(os.listdir(my_spark))
# print(sys.path)
# print("\n")

 Python Version:  3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
    Python Path:  C:\Users\d810216\AppData\Local\conda\conda\envs\my_root
 My Python Libs:  C:\Users\d810216/.myconfigs
   My Spark Dir:  C:\Users\d810216/spark2_dfanalysis
   My Spark Ctx:  <pyspark.sql.session.SparkSession object at 0x0000012AE005F5F8>


In [39]:
def pandas(df,n=5):
    return df.limit(5).toPandas()

In [3]:
# 6 columns: strings, ints, float
import random

num = 100000

lst_months = ['January','Feburary','March','April','May','June',
              'July','August','September','October','November','December']

indices = [int(x) for x in np.linspace(0,num-1,num)]
# print(indices)

years = [random.randint(1900,2019) for x in range(num)]
# print(years)

ages = [(2019 - x) for x in years]
# print(ages)

months = [random.choice(lst_months) for x in range(num)]
# print(months)

worth = [random.random() * 100000 for x in range(num)]
# print(worth)

lst_names = ['sarah','bill','steve','mary','alyssa','brian','elizabeth','josh','ryan','katie','connor',
             'erica','lisa','doug','stacie','chris','gary','tom','ellen','paula']
names = [random.choice(lst_names) for x in range(num)]


df = DFB.arrays_to_dataframe([indices,names,months,years,ages,worth],
                             ['index','name','month','year','age','net_worth'])


df.limit(4).toPandas()

,index,name,month,year,age,net_worth
0,0,brian,August,1913,106,"29,302.44"
1,1,lisa,November,1912,107,"55,594.58"
2,2,ryan,October,1987,32,"62,653.70"
3,3,doug,July,1910,109,"72,482.98"


# Multiple Dataframes
- df's 1-8

In [4]:
num = 250000

In [5]:
df1 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("integer",num=num,nrange=(1,28)),
     DFB.build_array("integer",num=num,nrange=(2010,2019)),
     DFB.build_array("integer",num=num,nrange=(0,23)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','month','day','year','hour','minute','second','price'])

In [6]:
df2 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("integer",num=num,nrange=(1,28)),
     DFB.build_array("integer",num=num,nrange=(2010,2019)),
     DFB.build_array("integer",num=num,nrange=(0,23)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','month','day','year','hour','minute','second','price'])

In [7]:
df3 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("integer",num=num,nrange=(1,28)),
     DFB.build_array("integer",num=num,nrange=(2010,2019)),
     DFB.build_array("integer",num=num,nrange=(0,23)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','month','day','year','hour','minute','second','price'])

In [8]:
df4 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("integer",num=num,nrange=(1,28)),
     DFB.build_array("integer",num=num,nrange=(2010,2019)),
     DFB.build_array("integer",num=num,nrange=(0,23)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','month','day','year','hour','minute','second','price'])

In [9]:
df5 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("integer",num=num,nrange=(1,28)),
     DFB.build_array("integer",num=num,nrange=(2010,2019)),
     DFB.build_array("integer",num=num,nrange=(0,23)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','month','day','year','hour','minute','second','price'])

In [10]:
df6 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("integer",num=num,nrange=(1,28)),
     DFB.build_array("integer",num=num,nrange=(2010,2019)),
     DFB.build_array("integer",num=num,nrange=(0,23)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','month','day','year','hour','minute','second','price'])

In [11]:
df7 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("integer",num=num,nrange=(1,28)),
     DFB.build_array("integer",num=num,nrange=(2010,2019)),
     DFB.build_array("integer",num=num,nrange=(0,23)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','month','day','year','hour','minute','second','price'])

In [12]:
df8 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("integer",num=num,nrange=(1,28)),
     DFB.build_array("integer",num=num,nrange=(2010,2019)),
     DFB.build_array("integer",num=num,nrange=(0,23)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','month','day','year','hour','minute','second','price'])

# Cache, Filter, Union.
# Filter, Union, Cache. 
- Filter, Union Cache - only a few iterations, but 5-10% faster

In [13]:
%%time

# cache, filter, append, count


for i,df in enumerate([df1,df2,df3,df4]):
    
    df.cache()
    df = df.filter(col("price") <= 50000)
    
    if i == 0:
        df_all = df
    else:
        df_all = df_all.union(df)
    
    #print(df_all.count())
    
print('df_all is now:',df_all.count())


# 100000
# 1st time:
# df_all is now: 197196
# Wall time: 17.4 s

# 2nd time: (quite typically)
# df_all is now: 198174
# Wall time: 546 ms

# 250000
# df_all is now: 494716
# Wall time: 20.2 s

df_all is now: 494716
Wall time: 20.2 s


In [14]:
%%time

# filter, append, cache, count

for i,df in enumerate([df5,df6,df7,df8]):
    
    df = df.filter(col("price") <= 50000)
    
    if i == 0:
        df_all = df
    else:
        df_all = df_all.union(df)
    
    #print(df_all.count())
    
df_all.cache()
print('df_all is now:',df_all.count())



# 1st time:
# df_all is now: 198248
# Wall time: 17.1 s

# 2nd time: (quite typically)
# df_all is now: 198350
# Wall time: 358 ms

# 250000
# df_all is now: 494921
# Wall time: 18.3 s

df_all is now: 494921
Wall time: 18.3 s


# Union df1 and df2, unpersist df1 or df2 ..
Seems that nothing happens ..

In [16]:
num = 100000

In [17]:
df10 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("integer",num=num,nrange=(1,28)),
     DFB.build_array("integer",num=num,nrange=(2010,2019)),
     DFB.build_array("integer",num=num,nrange=(0,23)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','month','day','year','hour','minute','second','price'])

In [18]:
df11 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("integer",num=num,nrange=(1,28)),
     DFB.build_array("integer",num=num,nrange=(2010,2019)),
     DFB.build_array("integer",num=num,nrange=(0,23)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','month','day','year','hour','minute','second','price'])

In [19]:
df12 = df10.union(df11)
print(df_all.count())

494921


In [20]:
df11.unpersist()

DataFrame[index: bigint, vin: string, division: bigint, month: bigint, day: bigint, year: bigint, hour: bigint, minute: bigint, second: bigint, price: double]

In [21]:
df11.count()

100000

In [22]:
df10.count()

100000

In [23]:
df12.count()

200000

# Check orderBy and join performance

In [45]:
num = 100000

In [46]:
df20 = DFB.arrays_to_dataframe(
    [DFB.build_array("integer",num=num,nrange=(1000000,2000000)),
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("integer",num=num,nrange=(1,28)),
     DFB.build_array("integer",num=num,nrange=(2010,2019)),
     DFB.build_array("integer",num=num,nrange=(0,23)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','month','day','year','hour','minute','second','price'])

In [56]:
df21 = DFB.arrays_to_dataframe(
    [DFB.build_array("integer",num=num,nrange=(1000000,2000000)),
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("integer",num=num,nrange=(1,28)),
     DFB.build_array("integer",num=num,nrange=(2010,2019)),
     DFB.build_array("integer",num=num,nrange=(0,23)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','month','day','year','hour','minute','second','price'])

In [57]:
# unique_id = [col('index'),lit('_'),col('division')]

In [58]:
# df22 = df20\
# .withColumn('key',concat(*unique_id))

In [59]:
# df23 = df21\
# .withColumn('key',concat(*unique_id))

In [60]:
df22 = df20.alias("a")\
.join(df21.alias("b"),col("a.index") == col("b.index"),"inner")

In [61]:
%%time
df22.count()

Wall time: 17.3 s


10046

In [70]:
df23 = df20.alias("a")\
.join(df21.alias("b"),col("a.index") == col("b.index"),"outer")

In [71]:
df23.count()

190957

In [67]:
df24 = df21.alias("a")\
.join(df20.alias("b"),col("a.index") == col("b.index"),"outer")

In [69]:
df24.count()

190957

In [73]:
df25 = df20.alias("a")\
.join(df21.alias("b"),col("a.index") == col("b.index"),"left_outer")

In [74]:
df25.count()

100507

In [75]:
df26 = df21.alias("a")\
.join(df20.alias("b"),col("a.index") == col("b.index"),"left_outer")

In [69]:
df26.count()

190957

In [38]:
df1.filter(col("price") <= 50000).count()

12368

In [39]:
df_order = df_all.orderBy("division").coalesce(24)

In [40]:
df_order.count()

49306

In [43]:
df_order.limit(8).toPandas()

,index,vin,division,month,day,year,hour,minute,second,price
0,3074,eec,100,7,11,2017,3,55,43,"32,840.12"
1,8000,srd,100,7,9,2019,5,24,50,"29,188.40"
2,8979,kic,100,7,21,2014,5,9,7,"14,472.40"
3,12945,wmq,100,5,13,2019,20,53,49,"6,441.38"


In [46]:
df_order1 = df_all.orderBy("division")

In [47]:
df_order1.count()

49306

In [48]:
df_order1.limit(8).toPandas()

,index,vin,division,month,day,year,hour,minute,second,price
0,18846,kkn,100,4,7,2016,10,5,27,"33,028.60"
1,8979,kic,100,7,21,2014,5,9,7,"14,472.40"
2,8000,srd,100,7,9,2019,5,24,50,"29,188.40"
3,22697,adf,100,12,11,2011,7,41,45,"38,874.78"


In [44]:
df_all.limit(8).toPandas()

,index,vin,division,month,day,year,hour,minute,second,price
0,1,zdk,809,2,11,2010,17,50,23,"1,271.38"
1,2,spo,631,3,20,2015,5,7,2,"6,543.39"
2,4,pxs,214,4,8,2017,0,58,48,"1,439.67"
3,5,dnb,473,10,17,2010,13,18,7,"19,084.29"


In [45]:
df1.limit(8).toPandas()

,index,vin,division,month,day,year,hour,minute,second,price
0,1,zdk,809,2,11,2010,17,50,23,"1,271.38"
1,2,spo,631,3,20,2015,5,7,2,"6,543.39"
2,3,gxk,432,11,16,2014,5,16,22,"52,792.71"
3,4,pxs,214,4,8,2017,0,58,48,"1,439.67"


# Working Example:

In [4]:
df.cache()

DataFrame[index: bigint, name: string, month: string, year: bigint, age: bigint, net_worth: double]

In [5]:
df.count()

100000

In [6]:
# 2 columns: integer, string

lst_cars = [random.choice(['Honda','Toyota','Chevy','Ford','Tesla','Volkswagon','Hyundai','Jeep']) for x in range(num)]

df2 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
    lst_cars],
    ['index','make'])

df2.limit(4).toPandas()

,index,make
0,1,Tesla
1,2,Volkswagon
2,3,Honda
3,4,Jeep


In [7]:
df2.count()

100000

In [8]:
df3 = df2

In [9]:
# 2 columns: integer, string
num = 25000

lst_cars = [random.choice(['Honda','Toyota','Chevy','Ford','Tesla','Volkswagon','Hyundai','Jeep']) for x in range(num)]

df5 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
    lst_cars],
    ['index','make'])

df5.limit(4).toPandas()

,index,make
0,1,Jeep
1,2,Volkswagon
2,3,Toyota
3,4,Tesla


In [10]:
df5.count()

25000

In [11]:
df6 = df5

In [12]:
df6.count()

25000

In [13]:
df2.unpersist()

DataFrame[index: bigint, make: string]

In [14]:
df2.count()

100000

In [15]:
df3.count()

100000